# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2025-04-17 ~
* 코드 작성 목적: 코사인 유사도 기반 추천 시스템 구현

# 1. 기초 설정

## 사용 라이브러리

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [ ]:
crawled = pd.read_csv('../data/dbpia_crawling_result.csv')

In [ ]:
df_cleaned = pd.read_csv("../data/cleaned_docs.csv", encoding='utf-8-sig')
df_cleaned.info()

In [ ]:
terms = [doc.split() for doc in df_cleaned['cleaned_doc']]
print(terms[:5])

# 2. 전처리

## B. TF-IDF 행렬 계산

# 3. 추천 시스템 구현

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

In [ ]:
title_to_index = dict(zip(crawled['제목'], crawled.index))
print(title_to_index)

In [ ]:
def PaperRecommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [idx[0] for idx in sim_scores]

    return crawled['제목'].iloc[movie_indices]

In [ ]:
PaperRecommendations('썸타기와 어장관리에 대한 철학적 고찰')

# 4. 평가